##  Challenge 3 – Strategy Optimization & Portfolio Selection

In this challenge, we move beyond just analyzing historical returns — our goal is to **optimize a simple trading strategy** using moving averages, assess risk, and make final portfolio decisions based on performance and drawdowns.

### Tasks:

1. **Find the best SMA** for the top 5 selected stocks  
   Backtest multiple SMA windows to determine which provides the most reliable trend-following signal.

2. **Create a bias strategy**  
   Define market bias using SMA crossovers or slope direction and execute trades based on that bias.

3. **Compare drawdowns**  
   Evaluate each stock's strategy by comparing **maximum drawdown**, **duration**, and **recovery** time.

4. **Pick the best 3 stocks** for your portfolio  
   Use performance and risk-adjusted metrics to finalize the best 3 candidates for long-term investing.


##  Task 1: Find Best SMA Window

We test SMA windows from 10 to 200 days. For each:
- Signal: **Buy when price > SMA**
- Hold no position otherwise
- Backtest using daily returns
- Select the window with **highest cumulative return**

This reveals the ideal SMA length for trend-following performance per stock.


In [67]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 


In [2]:
tickers = ['AAPL', 'HD', 'V', 'LMT', 'NEE']

stocks = yf.download(tickers)

C:\Users\alexm\AppData\Local\Temp\ipykernel_37496\1391946197.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stocks = yf.download(tickers)
[*********************100%***********************]  5 of 5 completed


In [3]:
close = stocks.loc[:,'Close'].copy().dropna()
close

Ticker,AAPL,HD,LMT,NEE,V
Date,,,,,
2008-03-19,3.897125,17.370304,59.418404,9.120476,12.514246
2008-03-20,4.005321,18.159857,59.028992,9.129474,14.252952
2008-03-24,4.193460,18.936472,59.459663,9.360448,13.229659
2008-03-25,4.237039,18.612885,60.179489,9.388947,14.009312
2008-03-26,4.359661,18.224577,59.725185,9.402444,14.166571
...,...,...,...,...,...
2025-06-18,196.580002,347.029999,468.600006,71.570000,340.380005
2025-06-20,201.000000,349.619995,470.559998,71.529999,338.570007
2025-06-23,201.500000,356.959991,472.459991,70.730003,343.750000


Function to determine the best SMA window for a single stock

In [ ]:
def find_best_sma_for_stock(close_series, sma_range=range(10, 201)): # This defines a stock's closing price series and tries SMA windowns from 10 to 201
    returns = close_series.pct_change().dropna() # calculates daily returns for each stock
    best_sma = None # initializes two variables to track the best SMA and its corresponding cumulative return 
    best_cum_return = -np.inf # ensures that any real return will be larger

    for window in sma_range:
        sma = close_series.rolling(window=window).mean() #calculates the sma for each window
        signal = (close_series > sma).shift(1)
        signal = signal.fillna(0).astype(int)
        strategy_returns = signal * returns
        cum_return = (1 + strategy_returns).prod()

        if cum_return > best_cum_return:
            best_cum_return = cum_return
            best_sma = window

    return best_sma, best_cum_return


Loop through each stock to find its optimal SMA and cumulative return

In [14]:
best_smas = {}

for ticker in close.columns:
    best_sma, cum_ret = find_best_sma_for_stock(close[ticker])
    best_smas[ticker] = {'Best SMA': best_sma, 'Cumulative Return': cum_ret}

best_smas_df = pd.DataFrame(best_smas).T
best_smas_df


,Best SMA,Cumulative Return
AAPL,19.0,39.016735
HD,187.0,10.743589
LMT,91.0,3.745859
NEE,191.0,2.957151
V,200.0,6.465713


Store results in a DataFrame

## Task 2: Apply Market Bias Using SMA Crossovers
We define bullish bias using 50-day and 200-day SMAs.
- Trade only when 50-SMA > 200-SMA


In [53]:
sma_s = 50
sma_l = 200

# Create two empty DataFrames to store the results
sma_50_df = pd.DataFrame(index=close.index)
sma_200_df = pd.DataFrame(index=close.index)

# Loop over each stock
for ticker in close.columns:
    sma_50_df[ticker] = close[ticker].rolling(window=sma_s).mean()
    sma_200_df[ticker] = close[ticker].rolling(window=sma_l).mean()


In [54]:
sma_50_df.dropna(inplace=True)
sma_200_df.dropna(inplace=True)

Create binary position signals (1 = trade, 0 = no trade)

In [55]:
# Create a new DataFrame to store positions
positions = pd.DataFrame(index=close.index)

# Loop through each stock
for ticker in close.columns:
    sma_s = sma_50_df[ticker].reindex(close.index) #reindex makes sure we have the same index, in this case the same date
    sma_l = sma_200_df[ticker].reindex(close.index) 
    
    # Long (1) if short SMA > long SMA, otherwise stay out (0)
    positions[ticker] = np.where(sma_s > sma_l, 1, 0)


Calculate log returns for Buy & Hold and Strategy

In [56]:
# Create empty DataFrames to store results
returns_bh = pd.DataFrame(index=close.index)       # Buy & Hold log returns
strategy_returns = pd.DataFrame(index=close.index) # Strategy returns (based on SMA bias)

# Loop through each stock
for ticker in close.columns:
    # Log returns for Buy & Hold
    returns_bh[ticker] = np.log(close[ticker] / close[ticker].shift(1))
    
    # Strategy returns based on shifted SMA bias signal
    strategy_returns[ticker] = positions[ticker].shift(1) * returns_bh[ticker]

Evaluate cumulative return and volatility for each strategy

In [58]:
import pandas as pd
import numpy as np

summary = []

for ticker in close.columns:
    bh = returns_bh[ticker].dropna()
    strat = strategy_returns[ticker].dropna()

    bh_cum = (1 + bh).prod() - 1
    strat_cum = (1 + strat).prod() - 1

    bh_vol = bh.std() * np.sqrt(252) # b&h standard deviation annual
    strat_vol = strat.std() * np.sqrt(252)
    
    summary.append({
        'Ticker': ticker,
        'Buy & Hold Cumulative Return': bh_cum,
        'Strategy Cumulative Return': strat_cum,
        'Buy & Hold Volatility': bh_vol,
        'Strategy Volatility': strat_vol,
           })

summary_df = pd.DataFrame(summary).set_index('Ticker')
summary_df




,Buy & Hold Cumulative Return,Strategy Cumulative Return,Buy & Hold Volatility,Strategy Volatility
Ticker,,,,
AAPL,21.613625,15.858418,0.308861,0.237311
HD,10.370103,2.855766,0.264148,0.201037
LMT,3.764627,2.004939,0.235543,0.172438
NEE,3.556962,1.964588,0.246806,0.182997
V,12.525864,8.667617,0.287260,0.212083


## Task 3: Drawdown Analysis
Assess downside risk by calculating max drawdown for Buy & Hold and the Strategy.


Calculate drawdowns for each strategy

In [63]:
def calculate_drawdown(series):
    cumulative = series.cumsum()
    peak = cumulative.cummax()
    drawdown = peak - cumulative
    return drawdown

# Create DataFrames to store drawdowns
drawdown_bh = pd.DataFrame(index=returns_bh.index)
drawdown_strategy = pd.DataFrame(index=strategy_returns.index)

# Loop through each stock
for ticker in close.columns:
    drawdown_bh[ticker] = calculate_drawdown(returns_bh[ticker])
    drawdown_strategy[ticker] = calculate_drawdown(strategy_returns[ticker])

# Calculate maximum drawdown per stock
max_dd_summary = pd.DataFrame({
    'Buy & Hold Max Drawdown': drawdown_bh.max(),
    'Strategy Max Drawdown': drawdown_strategy.max()
})

max_dd_summary


,Buy & Hold Max Drawdown,Strategy Max Drawdown
AAPL,0.887544,0.609041
HD,0.505480,0.477934
LMT,0.704570,0.488935
NEE,0.600116,0.440550
V,0.731985,0.452005


## Task 4: Pick Best 3 Stocks
We rank stocks based on Strategy Return / Strategy Max Drawdown.

In [66]:
# Create a scoring metric: Return-to-Drawdown Ratio
score = summary_df['Strategy Cumulative Return'] / max_dd_summary['Strategy Max Drawdown']

# Get the top 3 tickers based on this metric
top_3_stocks = score.sort_values(ascending=False).head(3)

# Show the result
print("Top 3 Stocks Based on Strategy Return-to-Drawdown Ratio:")
print(top_3_stocks)


Top 3 Stocks Based on Strategy Return-to-Drawdown Ratio:
Ticker
AAPL    26.038333
V       19.175917
HD       5.975229
dtype: float64
